## Supply Chain Optimization Problem

### Double Echelon, Non - Divisible Demand


You have two existing plants located at P ={Barrie,   Cambridge}.  Each  plant  manufactures a  single  product,  which  needs  to  be  distributed through a set of candidate warehouses to be located among J ={Thunder Bay,  Sudbury, Ottawa, Toronto, Kingston, Windsor, Niagara Falls, London} (not all potential warehouses will be  opened).  Each  of  the  potential  warehouses has  a  fixed  cost  to  operate and a  fixed  capacity.  Transportation has a cost per unit of product transported, and since transportation is performed by a 3PL, there is an unlimited capacity. The product produced at these plants is to be distributed to 20 retailers in different cities (of your choice) located all over Canada. Let R ={1,2,3,...,20} denotethe retailers. Each retailer has a different annual demand for this product denoted by dr, that is  randomly  generated  between  10,000 and 75,000 units.  Distances  between  facilities  and/or  retailers may be found using the existing highway network in Canada (e.g. ,  using Google maps). The goal is to minimize total cost. 

Note, the cost of traveling between plants, warehouses and retailers are pre-calculated, and the operating costs and capacities are known.


#### Mathematical model

Decision Variables:

$x_{ij}$: if group $i$ is sitting in row $j$.

$y{j}$: the surplus of seats in row $j$

$u$: total seated groups

$z$: the number of unseated groups

$m$: the total number of people sitting in the theatre


Sets :

$I$: The set of plants (1,2)  i  I 

$J$: The set of warehouses (1...8)  j  J

$R$: The set of retailers (1...20) r  R

Parameters

$d{r}$= demand of retailer r  R 

$P{i}$=production capacity for plant i I

$Q{j}$=capacity of warehouse j J

$f{j}$=fixed cost of operating warehouse j J

$C$ = cost of transportation per km per unit= $0.187/km-unit

$C{ij}$= transportation cost of a unit for the distance Dij for all i  I, j  J

$C{jr}$= transportation cost of a unit for the distance Djr for all  j  J,  r  R


Decision Variables:

$x{ij}$= quantity of product supplied from plant i  I to warehouse jJ

$y{jr}$= 1 if retailer rR has demand met by warehouse jJ, 0 otherwise

$Z{j}$ = 1 if we open a warehouse at location jJ, 0 otherwise

$$\begin{array}{rll}
\text{Minimize} & \displaystyle  C(p) = \sum_{i=1}^I\sum_{j= 1}^J x_{ij} * C_ij + \sum_{j=1}^J\sum_{r= 1}^R Y_{jr} * C_jr * d{r} + \sum_{j=1}^J f_{j} * Z_{j}  \\
\text{subject to:} & \displaystyle \sum_{j=1}^J\sum_{r= 1}^R y_{jr} = 1, \quad \forall r\in R\\
& \displaystyle \sum_{i=j}^J x_{ij} < P_{i},\quad \forall i\in I\\
& \displaystyle \sum_{r=1}^R (d_{r} * y_{jr})<= d_{r}*Y_{ij} \quad \forall j\in J\\
& \displaystyle \sum_{j= 1}^J Z_{j} <= J\\
& \displaystyle Z_{j} E(0,1) \quad \forall j\in J\\
& \displaystyle x_{ij} >=0 \quad \forall i\in I \quad \forall j\in J\\
& \displaystyle y_{jr} E(0,1) \quad \forall j\in J \quad \forall r\in R\\
\end{array}$$


In [2]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum
from random import random
import numpy as np
import random

In [3]:
m = gp.Model()

Academic license - for non-commercial use only - expires 2021-08-07
Using license file /Users/meltran/gurobi.lic


In [4]:
I=2
R=20
J=8

In [4]:

d = [10000,15000,20000,12000,25000,30000,35000,13000,14000,16000,40000,38000,28000,19000,21000,31000,23000,71000,62000,16000]
P = [1500000,1500000]
# Qj,fj
q = [200000,400000,200000,800000,800000,400000,300000,600000]
f = [90000,110000,150000,150000,90000,110000,110000,150000]
# define cambrage as D1 and Barrie as D2
dij = [1446,447,479,97,343,277,119,100],[1291,292,412,111,343,432,208,255]
djr = [[1444,1462,1396,1267,1399,1428,1423,1436,1480,1556,1631,1463,1410,1373,1382,1462,782,1007,1391,1390],[445,463,397,722,400,429,493,437,481,557,632,484,499,545,383,474,299,0,392,391],[477,495,271,754,449,508,576,517,560,269,197,0,531,577,377,553,719,484,424,471],[95,113,139,369,0,60,143,69,112,189,263,450,149,192,30,105,708,400,45,28],[342,359,181,618,262,321,389,330,373,83,0,196,396,442,255,366,809,632,292,284],[297,291,489,0,369,319,232,305,361,543,618,753,253,191,381,271,1029,721,350,349],[126,144,269,378,128,72,151,72,21,315,390,577,180,201,161,114,805,497,128,109],[119,113,312,191,191,142,54,128,184,366,441,576,61,0,204,94,852,544,172,171]]

cij= [[270.402, 83.589, 89.573, 18.139, 64.141, 51.799, 22.253, 18.7], [241.417, 54.604, 77.044, 20.757, 64.141, 80.784 , 38.896, 47.685]]
cjr =np.dot(0.18,djr)

d=np.array(d) 
P=np.array(P)
q=np.array(q)
f=np.array(f)
cij=np.array(cij)
cjr=np.array(cjr)


print (cij[1][5])


80.784


In [5]:
xij={}
for i in range(I):
    for j in range(J):
        xij[i,j] = m.addVar(vtype=GRB.INTEGER,lb=0,name="xij_"+str(i)+str(j))
            
z={}
for j in range(J):
    z[j] = m.addVar(vtype=GRB.BINARY, name="z_"+str(j))
    
tjr={}
for j in range(J):
    for r in range(R):
        tjr[j,r] = m.addVar(vtype=GRB.INTEGER,lb=0,name="tjr_"+str(j)+str(r))
        
        
#u=m.addVar(vtype=GRB.INTEGER,lb=0, ub=I,name="u")
t=m.addVar(vtype=GRB.INTEGER,lb=0, name="t")
p=m.addVar(vtype=GRB.INTEGER,lb=0, name="p")
g=m.addVar(vtype=GRB.INTEGER,lb=0, name="g")
o=m.addVar(vtype=GRB.INTEGER,lb=0, name="o")


In [6]:
m.setObjective(sum(f[j]*z[j] for j in range(J)) + t + p, GRB.MINIMIZE)

In [7]:

#constraints
m.addConstr(sum(z[j] for j in range(J))<= J) 
m.addConstrs((sum(tjr[j,r] for j in range(J))== d[r]) for r in range(R))
m.addConstrs((sum(xij[i,j] for j in range(J))<= P[i]) for i in range(I))
m.addConstrs((sum(xij[i,j] for i in range(I))<= q[j]*z[j]) for j in range(J))   

m.addConstrs((sum(xij[i,j] for i in range(I)) == sum(tjr[j,r] for r in range(R)) for j in range(J)))   

#objective function constraints
m.addConstr(quicksum(tjr[j,r]*cjr[j][r] for j in range(J) for r in range(R)) ==t)
m.addConstr(quicksum(xij[i,j]*cij[i][j] for i in range(I) for j in range(J))==p) 


<gurobi.Constr *Awaiting Model Update*>

In [8]:
#m.Params.TimeLimit=300 #(seconds) optional: sets a time limit for optimization only if you need to prematurely stop the solution procedure
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 41 rows, 188 columns and 556 nonzeros
Model fingerprint: 0x7819b8fc
Variable types: 0 continuous, 188 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+05]
  Objective range  [1e+00, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 2e+06]
Found heuristic solution: objective 3.696719e+07
Presolve removed 1 rows and 2 columns
Presolve time: 0.01s
Presolved: 40 rows, 186 columns, 533 nonzeros
Variable types: 0 continuous, 186 integer (8 binary)

Root relaxation: objective 2.313783e+07, 48 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2.3138e+07    0    7 3.6967e+07 2.3138e+07  37.4%     -    0s
H    0     0                    2.384347e+07 2.3138e+07  2.96%

In [9]:
for myVars in m.getVars():
    print('%s %g' % (myVars.varName, myVars.x))
    
#https://docs.python.org/2.4/lib/typesseq-strings.html

xij_00 -0
xij_01 -0
xij_02 -0
xij_03 268000
xij_04 -0
xij_05 0
xij_06 14000
xij_07 125000
xij_10 -0
xij_11 94000
xij_12 38000
xij_13 -0
xij_14 0
xij_15 -0
xij_16 -0
xij_17 -0
z_0 -0
z_1 1
z_2 1
z_3 1
z_4 0
z_5 0
z_6 1
z_7 1
tjr_00 -0
tjr_01 -0
tjr_02 -0
tjr_03 -0
tjr_04 -0
tjr_05 -0
tjr_06 -0
tjr_07 -0
tjr_08 -0
tjr_09 -0
tjr_010 -0
tjr_011 -0
tjr_012 -0
tjr_013 -0
tjr_014 -0
tjr_015 -0
tjr_016 0
tjr_017 -0
tjr_018 -0
tjr_019 -0
tjr_10 -0
tjr_11 -0
tjr_12 -0
tjr_13 -0
tjr_14 -0
tjr_15 -0
tjr_16 -0
tjr_17 -0
tjr_18 -0
tjr_19 -0
tjr_110 -0
tjr_111 -0
tjr_112 -0
tjr_113 -0
tjr_114 -0
tjr_115 -0
tjr_116 23000
tjr_117 71000
tjr_118 -0
tjr_119 -0
tjr_20 -0
tjr_21 -0
tjr_22 -0
tjr_23 -0
tjr_24 -0
tjr_25 -0
tjr_26 -0
tjr_27 -0
tjr_28 -0
tjr_29 -0
tjr_210 -0
tjr_211 38000
tjr_212 -0
tjr_213 -0
tjr_214 -0
tjr_215 -0
tjr_216 -0
tjr_217 -0
tjr_218 -0
tjr_219 -0
tjr_30 10000
tjr_31 15000
tjr_32 20000
tjr_33 -0
tjr_34 25000
tjr_35 30000
tjr_36 -0
tjr_37 13000
tjr_38 0
tjr_39 16000
tjr_310 40000
tjr_

In [10]:
print('\nObjective Function Value: %g' %m.objVal) # prints objective function value
print('')
print('Solution:')


#for j in range(J):
#    print('For row %s:' %j)
#    print('total empty seats: %s' %y[j].x)
#    
#    for i in range(I):
#        print(' Group %s sits %g' %(i,x[i,j].x)) # prints out number of operation j in each room i

   


Objective Function Value: 2.37124e+07

Solution:


In [11]:
m.write("checkPartA.lp")